In [1]:
import sys
import os
sys.path.append('../')
import numpy as np
import scipy.signal as sps
from source.reader import read_raw
from source.interference_reject import InterferenceReject
import plotly  as py
import plotly.graph_objects as go
%matplotlib inline

In [2]:
# Data import
file_name = "/Users/had-mr/github/data_interference/fid_shield/ratio_1_4coils.h5"
data, index, metadata = read_raw(file_name)
num_channels_all = metadata['console']['adc']['number_channels']
num_channels_signal = metadata['rf']['receive']['num_signal_channels']
matrix_size = metadata['sequence']['encoding']['matrix_size']
acq_bandwidth = metadata['sequence']['readout']['readouts'][0]['bandwidth']

In [3]:
# correct prescan data
select_signal_channels=[0, 1]
interf_rej_1 = InterferenceReject(channels_signal=select_signal_channels,channels_noise=[3,])
interf_rej_2 = InterferenceReject(channels_signal=select_signal_channels,channels_noise=[3, 4])
interf_rej_3 = InterferenceReject(channels_signal=select_signal_channels,channels_noise=[3, 4, 5])
data_raw = data['noise']
data_cor_1 = interf_rej_1.apply(data_signal=data_raw, data_noise=data_raw)
data_cor_2 = interf_rej_2.apply(data_signal=data_raw, data_noise=data_raw)
data_cor_3 = interf_rej_3.apply(data_signal=data_raw, data_noise=data_raw)

## Plot noise density (prescan)

In [12]:
from source.utils import compute_spectrogram
axis_pow, pow_raw = compute_spectrogram(data_raw[select_signal_channels, :, :], acq_bandwidth)
axis_pow, pow_cor_1 = compute_spectrogram(data_cor_1, acq_bandwidth)
axis_pow, pow_cor_2 = compute_spectrogram(data_cor_2, acq_bandwidth)
axis_pow, pow_cor_3 = compute_spectrogram(data_cor_3, acq_bandwidth)

ch = 0

# average noise in 10 k BW
sel_passband = np.abs(axis_pow) < 5e3
sel_stopband = np.abs(axis_pow) > 20e3
print("Average raw = {} dB".format(10 * np.log10(np.mean(pow_raw[ch, sel_passband]))))
print("Average baseline = {} dB".format(10 * np.log10(np.mean(pow_raw[ch, sel_stopband]))))
print("Average 1 channel = {} dB".format(10 * np.log10(np.mean(pow_cor_1[ch, sel_passband]))))
print("Average 2 channel = {} dB".format(10 * np.log10(np.mean(pow_cor_2[ch, sel_passband]))))
print("Average 3 channel = {} dB".format(10 * np.log10(np.mean(pow_cor_3[ch, sel_passband]))))

# Interactive plot
fig = go.Figure()
fig.update_layout(plot_bgcolor="white")
line_style = {'color': 'orange', 'width': 2}
fig.add_trace(go.Scatter(x=axis_pow, y=10 * np.log10(pow_raw[ch, :]), line=line_style, name='Raw'))
line_style = {'color': 'gray', 'width': 2}
fig.add_trace(go.Scatter(x=axis_pow, y=10 * np.log10(pow_cor_1[ch, :]), line=line_style, name='1 reference coil'))
line_style = {'color': 'green', 'width': 2}
fig.add_trace(go.Scatter(x=axis_pow, y=10 * np.log10(pow_cor_2[ch, :]), line=line_style, name='2 reference coils'))
line_style = {'color': 'blue', 'width': 2}
fig.add_trace(go.Scatter(x=axis_pow, y=10 * np.log10(pow_cor_3[ch, :]), line=line_style, name='3 reference coils'))
fig.show()


Average raw = -10.79568144401823 dB
Average baseline = -35.65736039805981 dB
Average 1 channel = -32.45951510018618 dB
Average 2 channel = -33.62039381735458 dB
Average 3 channel = -34.15848815354628 dB
